In [1]:
from matplotlib import pyplot as plt
import xarray as xr
import numpy as np
import dask
from dask.diagnostics import progress
from tqdm.autonotebook import tqdm 
import intake
import fsspec
import seaborn as sns
import math
import cartopy.crs as ccrs
import cartopy.feature as cf
import os
import cdsapi

%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

<ipython-input-1-fcc32bc5f501>:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [28]:
climatology = xr.open_dataset('/nobackup/users/lulur/Daily_hist_temps/air.2m.day.1981-2010.ltm.nc')
climatology.coords['dayofyear'] = climatology.time.dt.dayofyear
climatology_doy = climatology.drop('time')

/home/lulur/.conda/envs/cicoes39/lib/python3.9/site-packages/xarray/coding/times.py:527: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/lulur/.conda/envs/cicoes39/lib/python3.9/site-packages/xarray/coding/times.py:119: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/home/lulur/.conda/envs/cicoes39/lib/python3.9/site-packages/numpy/core/_asarray.py:102: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime o

In [44]:
climatology

/home/lulur/.conda/envs/cicoes39/lib/python3.9/site-packages/xarray/coding/times.py:119: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


<xarray.Dataset>
Dimensions:             (lat: 94, lon: 192, nbnds: 2, time: 365)
Coordinates:
  * lat                 (lat) float32 88.54 86.65 84.75 ... -84.75 -86.65 -88.54
  * lon                 (lon) float32 0.0 1.875 3.75 5.625 ... 354.4 356.2 358.1
  * time                (time) object 0001-01-01 00:00:00 ... 0001-12-31 00:0...
    dayofyear           (time) int64 1 2 3 4 5 6 7 ... 360 361 362 363 364 365
Dimensions without coordinates: nbnds
Data variables:
    climatology_bounds  (time, nbnds) datetime64[ns] ...
    air                 (time, lat, lon) float32 ...
    valid_yr_count      (time, lat, lon) float32 ...
Attributes:
    Conventions:                    COARDS
    title:                          mean daily NMC reanalysis (1981)
    description:                    Data is from NMC initialized reanalysis\n...
    platform:                       Model
    not_missing_threshold_percent:  minimum 3% values input to have non-missi...
    history:                        Created 2011/06/28 by doDayLTM\nConverted...
    dataset_title:                  NCEP-NCAR Reanalysis 1
    References:                     http://www.psl.noaa.gov/data/gridded/data...

In [2]:
def load_temps(year):
    dataset = xr.open_dataset('/nobackup/users/lulur/Daily_hist_temps/air.2m.gauss.'+str(year)+'.nc')
    return dataset

In [32]:
def find_anomalies(year):
    daily_temps = load_temps(year)
    daily_temps.coords['dayofyear'] = daily_temps.time.dt.dayofyear
    daily_temps_doy = daily_temps.drop('time')
    daily_temps['anomalies'] = daily_temps_doy['air']-climatology_doy['air']
    return daily_temps

In [50]:
year = 1980
daily_temps = load_temps(year)
daily_temps.coords['dayofyear'] = daily_temps.time.dt.dayofyear
daily_temps_doy = daily_temps.drop('time')
daily_temps['anomalies'] = daily_temps_doy['air']
daily_temps
#data1980 = find_anomalies(1984)
# daily_temps.coords['dayofyear'] = daily_temps.time.dt.dayofyear
# print(len(daily_temps['dayofyear']))
# daily_temps_doy = daily_temps.drop('time')
# air = daily_temps_doy.air
# print(air.shape)
# air.where(air.dayofyear!=air.dayofyear[60], drop=True)

<xarray.Dataset>
Dimensions:    (lat: 94, lon: 192, time: 366)
Coordinates:
  * time       (time) datetime64[ns] 1980-01-01 1980-01-02 ... 1980-12-31
  * lat        (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon        (lon) float32 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365 366
Data variables:
    air        (time, lat, lon) float32 ...
    anomalies  (time, lat, lon) float32 ...
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1980)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [ ]:
air

In [42]:
data1979 = find_anomalies(1979)
data1980 = find_anomalies(1980)
histdata = xr.concat((data1979,data1980),dim='time')
years = np.arange(1981,2020)
for year in years:
    data_yr = find_anomalies(year)
    histdata = xr.concat((histdata,data_yr),dim='time')    

ValueError: arguments without labels along dimension 'time' cannot be aligned because they have different dimension sizes: {365, 366}

In [35]:
histdata

<xarray.Dataset>
Dimensions:  (lat: 94, lon: 192, time: 14975)
Coordinates:
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2019-12-31
Data variables:
    air      (time, lat, lon) float32 239.6 239.6 239.5 ... 251.8 251.8 251.8
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1979)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [10]:
hist_daily_anomalies = histdata.groupby('time.day')-climatology
hist_daily_anomalies

ValueError: incompatible dimensions for a grouped binary operation: the group variable 'day' is not a dimension on the other argument

In [4]:
data1979

<xarray.Dataset>
Dimensions:  (lat: 94, lon: 192, time: 365)
Coordinates:
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 1979-12-31
Data variables:
    air      (time, lat, lon) float32 ...
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1979)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/03/13 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...